In [5]:
%reset -f
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import random
import shap
import matplotlib.colors as colors
import matplotlib as mpl
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cartopy.feature as cfeature 
import cartopy.mpl.ticker as cticker 

import imageio
import os
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from xgboost import plot_importance, plot_tree


In [6]:
matplotlib.rcParams['font.family'] = 'Times New Roman'

# Read the data (OISST & Argo)
- uniform the time dim

In [7]:
oisst = xr.open_dataset(r"C:\Users\lv299\OneDrive\STF_Paper\Subtropical-Front-Modeling-Code\Step1\OISST_Merge_Cut_1_4Deg_Monthly\OISST_1_4Deg_monthly.nc")
argo = xr.open_dataset(r"C:\Users\lv299\OneDrive\STF_Paper\Subtropical-Front-Modeling-Code\Step1\Argo_Merge_Cut_1_4Deg\Argo_Interpolate\argo_1_4_deg.nc")

In [8]:
oisst['time'] = argo.time
oisst = oisst.sst

In [9]:
argo = argo.sst

In [10]:
argo.name = 'st100' 


In [11]:
xgb_merge = xr.merge([oisst, argo])
xgb_merge

<xarray.Dataset>
Dimensions:  (time: 240, lat: 82, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 2004-01-15 2004-02-15 ... 2023-12-15
  * lat      (lat) float64 -50.12 -49.88 -49.62 -49.38 ... -30.38 -30.12 -29.88
  * lon      (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
Data variables:
    sst      (time, lat, lon) float64 ...
    st100    (time, lat, lon) float64 ...

# Flatten the input data into a pd dataframe

- also add the day and year into the pd dataframe

In [12]:
def flatten_day(data):
    """
    Flattens the data into a pd dataframe
    Add a day column and a year column to the data
    Delete the time column
    
    Parameters:
    -----------
    data1: xarray dataset
        The first dataset to be flattened
    data2: xarray dataset
        The second dataset to be flattened
    
    return: pd dataframe
        The flattened pd dataframe
    """

    pd_data = data.to_dataframe().reset_index()

    base_date = pd.to_datetime('2004-01-01')
    
    pd_data['day'] = (pd_data['time'] - base_date).dt.days

    pd_data['year'] = pd_data['time'].dt.year

    pd_data = pd_data.drop(columns = ['time'])

    return pd_data

# Split the training and testing data
- Randomly split the training and testing data by year
- Delete NaN data

In [26]:
def training_testing_split(data):
    """
    Split the data into training and testing data
    
    Parameters:
    -----------
    data: pd dataframe
        The data to be split
    
    return: pd dataframe, pd dataframe
        The training and testing data
    """
    years = list(range(2004, 2024))
    shuffled_years = pd.Series(years).sample(frac=1).tolist()
    training_years = shuffled_years[:15]
    testing_years = shuffled_years[15:]

    training_data = data[data['year'].isin(training_years)].dropna()
    testing_data = data[data['year'].isin(testing_years)].dropna()


    training_X = training_data.drop(columns = ['st100', 'year'], axis = 1)
    training_y = training_data['st100']
    testing_X = testing_data.drop(columns = ['st100', 'year'], axis = 1)
    testing_y = testing_data['st100']
    
    return training_X, testing_X, training_y, testing_y

# XGBoost build

In [36]:
def run_model(xgb_reg, training_X, testing_X, training_y, testing_y):
    """
    Train the model and return the model and the prediction
    
    Parameters:
    -----------
    xgb_reg: xgboost model
        The xgboost model to be trained
    training_X: pd dataframe
        The training data
    testing_X: pd dataframe
        The testing data
    training_y: pd dataframe
        The training label
    testing_y: pd dataframe
        The testing label
    
    return: xgboost model, np array
        The trained model and the prediction
    """
    xgb_model = xgb_reg.fit(training_X, training_y)
    y_pred = xgb_reg.predict(testing_X)
    rmse = np.sqrt(mean_squared_error(testing_y, y_pred))
    r2 = r2_score(testing_y, y_pred)
    print('RMSE: ', rmse)
    print('R^2: ', r2)

    return y_pred, xgb_model

In [38]:
params = {
    'learning_rate': 0.01,
    'n_estimators': 350,
    'max_depth': 10,
    'min_child_weight': 10,
    'subsample': 0.55,
    'colsample_bytree': 1,
    'objective': 'reg:squarederror',
    'n_jobs': -1,
    'tree_method': 'hist',
    'device': 'cuda'
}

In [33]:
pd_xgb_data = flatten_day(xgb_merge)
training_X, testing_X, training_y, testing_y = training_testing_split(pd_xgb_data)

In [40]:
xgb_reg = xgb.XGBRegressor(**params, seed = 42)
y_pred, xgb_model = run_model(xgb_reg, training_X, testing_X, training_y, testing_y)

RMSE:  0.6871439921305028
R^2:  0.972782629696578
